# Network Traffic Forecasting with _AutoTS_

In telco, accurate forecast of KPIs (e.g. network traffic, utilizations, user experience, etc.) for communication networks ( 2G/3G/4G/5G/wired) can help predict network failures, allocate resource, or save energy. 

In this notebook, we demostrate a reference use case where we use the network traffic KPI(s) in the past to predict traffic KPI(s) in the future. We demostrate how to use `AutoTS` in project [Zouwu](https://github.com/intel-analytics/analytics-zoo/tree/master/pyzoo/zoo/zouwu) to do time series forecasting in an automated and distributed way.

For demonstration, we use the publicly available network traffic data repository maintained by the [WIDE project](http://mawi.wide.ad.jp/mawi/) and in particular, the network traffic traces aggregated every 2 hours (i.e. AverageRate in Mbps/Gbps and Total Bytes) in year 2018 and 2019 at the transit link of WIDE to the upstream ISP ([dataset link](http://mawi.wide.ad.jp/~agurim/dataset/)). 

## Helper functions

This section defines some helper functions to be used in the following procedures. You can refer to it later when they're used.

In [1]:
def get_drop_dates_and_len(df, allow_missing_num=3):
    """
    Find missing values and get records to drop
    """
    missing_num = df.total.isnull().astype(int).groupby(df.total.notnull().astype(int).cumsum()).sum()
    drop_missing_num = missing_num[missing_num > allow_missing_num]
    drop_datetimes = df.iloc[drop_missing_num.index].index
    drop_len = drop_missing_num.values
    return drop_datetimes, drop_len

In [2]:
def rm_missing_weeks(start_dts, missing_lens, df):
    """
    Drop weeks that contains more than 3 consecutive missing values.
    If consecutive missing values across weeks, we remove all the weeks.
    """ 
    for start_time, l in zip(start_dts, missing_lens):
        start = start_time - pd.Timedelta(days=start_time.dayofweek)
        start = start.replace(hour=0, minute=0, second=0)
        start_week_end = start + pd.Timedelta(days=6)
        start_week_end = start_week_end.replace(hour=22, minute=0, second=0)

        end_time = start_time + l*pd.Timedelta(hours=2)
        if start_week_end < end_time:
            end = end_time + pd.Timedelta(days=6-end_time.dayofweek)
            end = end.replace(hour=22, minute=0, second=0)
        else:
            end = start_week_end
        df = df.drop(df[start:end].index)
    return df

In [3]:
# plot the predicted values and actual values (for the test data)
def plot_result(test_df, pred_df, dt_col="datetime", value_col="AvgRate", look_back=1):
    # target column of dataframe is "value"
    # past sequence length is 50
    pred_value = pred_df[value_col].values
    true_value = test_df[value_col].values[look_back:]
    fig, axs = plt.subplots(figsize=(12, 5))

    axs.plot(pred_df[dt_col], pred_value, color='red', label='predicted values')
    axs.plot(test_df[dt_col][look_back:], true_value, color='blue', label='actual values')
    axs.set_title('the predicted values and actual values (for the test data)')

    plt.xlabel(dt_col)
    plt.xticks(rotation=45)
    plt.ylabel(value_col)
    plt.legend(loc='upper left')
    plt.show()

## Download raw dataset and load into dataframe

Now we download the dataset and load it into a pandas dataframe. Steps are as below. 

* First, run the script `get_data.sh` to download the raw data. It will download the monthly aggregated traffic data in year 2018 and 2019 into `data` folder. The raw data contains aggregated network traffic (average MBPs and total bytes) as well as other metrics. 

* Second, run `extract_data.sh` to extract relavant traffic KPI's from raw data, i.e. `AvgRate` for average use rate, and `total` for total bytes. The script will extract the KPI's with timestamps into `data/data.csv`.

* Finally, use pandas to load `data/data.csv` into a dataframe as shown below

In [4]:
import os 
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
#%matplotlib inline

In [5]:
raw_df = pd.read_csv("data/data.csv")

Below are some example records of the data

In [6]:
raw_df.head()

,StartTime,EndTime,AvgRate,total
0,2018/01/01 00:00:00,2018/01/01 02:00:00,306.23Mbps,275605455598
1,2018/01/01 02:00:00,2018/01/01 04:00:00,285.03Mbps,256527692256
2,2018/01/01 04:00:00,2018/01/01 06:00:00,247.39Mbps,222652190823
3,2018/01/01 06:00:00,2018/01/01 08:00:00,211.55Mbps,190396029658
4,2018/01/01 08:00:00,2018/01/01 10:00:00,234.82Mbps,211340468977


## Data pre-processing

Now we need to do data cleaning and preprocessing on the raw data. Note that this part could vary for different dataset. 

For the network traffic data we're using, the processing contains 3 parts:
1. Convert string datetime to TimeStamp
2. Unify the measurement scale for `AvgRate` value - some uses Mbps, some uses Gbps 
3. Handle missing data (fill or drop).

In [7]:
df = pd.DataFrame(pd.to_datetime(raw_df.StartTime))

In [8]:
# we can find 'AvgRate' is of two scales: 'Mbps' and 'Gbps' 
raw_df.AvgRate.str[-4:].unique()

array(['Mbps', 'Gbps'], dtype=object)

In [9]:
# Unify AvgRate value
df['AvgRate'] = raw_df.AvgRate.apply(lambda x: float(x[:-4]) if x.endswith("Mbps") else float(x[:-4]) * 1000)

In [10]:
df["total"] = raw_df["total"]
df.set_index("StartTime", inplace=True)

In [11]:
full_idx = pd.date_range(start=df.index.min(), end=df.index.max(), freq='2H')
df = df.reindex(full_idx)
print("no. of n/a values:")
print(df.isna().sum())

no. of n/a values:
AvgRate    3
total      3
dtype: int64


Here, we drop weeks with more than 3 consecutive missing values and fill other missing values remained.

In [12]:
drop_dts, drop_len = get_drop_dates_and_len(df)
df = rm_missing_weeks(drop_dts, drop_len, df)

In [13]:
df.ffill(inplace=True)

In [14]:
# AutoTS requires input data frame with a datetime column
df.index.name = "datetime"
df = df.reset_index()

In [15]:
df.head()

,datetime,AvgRate,total
0,2018-01-01 00:00:00,306.23,2.756055e+11
1,2018-01-01 02:00:00,285.03,2.565277e+11
2,2018-01-01 04:00:00,247.39,2.226522e+11
3,2018-01-01 06:00:00,211.55,1.903960e+11
4,2018-01-01 08:00:00,234.82,2.113405e+11


In [16]:
df.describe()

,AvgRate,total
count,8760.000000,8.760000e+03
mean,454.050030,4.084920e+11
std,238.377074,2.146655e+11
min,86.490000,8.641890e+09
25%,273.715000,2.459111e+11
50%,410.590000,3.694360e+11
75%,558.200000,5.023054e+11
max,1760.000000,1.585238e+12


Plot the data to see how the KPI's look like

In [17]:
#ax = df.plot(y='AvgRate',figsize=(12,5), title="AvgRate of network traffic data")

In [18]:
#ax = df.plot(y='total',figsize=(12,5), title="total bytes of network traffic data")

## Time series forecasting with _AutoTS_

_AutoTS_ provides AutoML support for building end-to-end time series analysis pipelines (including automatic feature generation, model selection and hyperparameter tuning).

The general workflow using automated training contains below two steps. 
   1. create a ```AutoTSTrainer``` to train a ```TSPipeline```, save it to file to use later or elsewhere if you wish.
   2. use ```TSPipeline``` to do prediction, evaluation, and incremental fitting as well. 


First, you need to initialize RayOnSpark before using auto training (i.e. ```AutoTSTrainer```), and stop it after training finished. (Note RayOnSpark is not needed if you just use `TSPipeline` for inference, evaluation or incremental training.)


In [20]:
# init RayOnSpark in local mode
from zoo.common.nncontext import *
sc = getOrCreateSparkContext()
sc.stop()
from zoo import init_spark_on_local
from zoo.ray import RayContext
sc = init_spark_on_local(cores=4, spark_log_level="INFO")
ray_ctx = RayContext(sc=sc, object_store_memory="1g")
ray_ctx.init()

Current pyspark location is : /home/ding/Downloads/spark-2.4.3-bin-hadoop2.7/python/pyspark/__init__.py
Start to getOrCreate SparkContext
Successfully got a SparkContext


2020-09-29 16:03:58,851	INFO resource_spec.py:212 -- Starting Ray with 21.14 GiB memory available for workers and up to 0.93 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-09-29 16:03:59,082	WARNING services.py:923 -- Redis failed to start, retrying now.
2020-09-29 16:03:59,333	INFO services.py:1165 -- View the Ray dashboard at localhost:8265


{'node_ip_address': '10.241.98.64', 'raylet_ip_address': '10.241.98.64', 'redis_address': '10.241.98.64:6379', 'object_store_address': '/tmp/ray/session_2020-09-29_16-03-58_850452_17650/sockets/plasma_store', 'raylet_socket_name': '/tmp/ray/session_2020-09-29_16-03-58_850452_17650/sockets/raylet', 'webui_url': 'localhost:8265', 'session_dir': '/tmp/ray/session_2020-09-29_16-03-58_850452_17650'}


{'node_ip_address': '10.241.98.64',
 'raylet_ip_address': '10.241.98.64',
 'redis_address': '10.241.98.64:6379',
 'object_store_address': '/tmp/ray/session_2020-09-29_16-03-58_850452_17650/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-09-29_16-03-58_850452_17650/sockets/raylet',
 'webui_url': 'localhost:8265',
 'session_dir': '/tmp/ray/session_2020-09-29_16-03-58_850452_17650'}

Then we initialize a `AutoTSTrainer`.
* `dt_col`: the column specifying datetime.
* `target_col`: target column to predict. Here, we take `AvgRate` KPI as an example.
* `horizon` : num of steps to look forward. 
* `extra_feature_col`: a list of columns which are also included in input data frame as features except target column.

In [23]:
from zoo.zouwu.autots.forecast import AutoTSTrainer

trainer = AutoTSTrainer(dt_col="datetime",
                        target_col="AvgRate",
                        horizon=1,
                        extra_features_col=None)

We can set some searching presets such as `look_back` which indicates the history time period we want to use for forecasting.
lookback can be an int which it is a fixed values, or can be a tuple to indicate the range for sampling.

In [24]:
# look back in range from one week to 3 days to predict the next 2h.
look_back = (36, 84)

We need to split the data frame into train, validation and test data frame before training. You can use `train_val_test_split` as an easy way to finish it.

In [25]:
from zoo.automl.common.util import train_val_test_split
train_df, val_df, test_df = train_val_test_split(df, 
                                                 val_ratio=0.1, 
                                                 test_ratio=0.1,
                                                 look_back=look_back[0])

Then we fit on train data and validation data. 

You can use `recipe` to specify searching method as well as other searching presets such as stop criteria .etc. The `GridRandomRecipe` here is a recipe that combines grid search with random search to find the best set of parameters. For more details, please refer to analytics-zoo document [here](https://analytics-zoo.github.io/master/#APIGuide/AutoML/recipe/).

In [26]:
from zoo.automl.config.recipe import LSTMGridRandomRecipe

In [27]:
%%time
ts_pipeline = trainer.fit(train_df, val_df, 
                          recipe=LSTMGridRandomRecipe(
                              num_rand_samples=1,
                              epochs=1,
                              look_back=look_back, 
                              batch_size=[64]),
                          metric="mse")

Trial name,status,loc,batch_size,dropout_2,lr,lstm_1_units,lstm_2_units,past_seq_len,selected_features,iter,total time (s)
train_func_74456_00000,TERMINATED,,64,0.309508,0.00853517,16,16,38,"[""MONTH(datetime)"", ""DAY(datetime)"", ""IS_AWAKE(datetime)""]",10,50.5247
train_func_74456_00001,TERMINATED,,64,0.471959,0.00333704,128,32,44,"[""MONTH(datetime)"", ""WEEKDAY(datetime)"", ""IS_BUSY_HOURS(datetime)"", ""IS_WEEKEND(datetime)""]",10,69.7632
train_func_74456_00002,TERMINATED,,64,0.402179,0.00841629,32,64,50,"[""HOUR(datetime)"", ""DAY(datetime)"", ""IS_BUSY_HOURS(datetime)"", ""IS_AWAKE(datetime)"", ""IS_WEEKEND(datetime)""]",10,49.5204


best log dir is  /home/ding/ray_results/automl/train_func_0_batch_size=64,dropout_2=0.30951,lr=0.0085352,lstm_1_units=16,lstm_2_units=16,past_seq_len=38,selected_features=["MONT_2020-09-29_16-06-49k6nmgnt8
The best configurations are:
selected_features : ["MONTH(datetime)", "DAY(datetime)", "IS_AWAKE(datetime)"]
model : LSTM
lstm_1_units : 16
dropout_1 : 0.2
lstm_2_units : 16
dropout_2 : 0.3095076432762375
lr : 0.008535167820304366
batch_size : 64
epochs : 1
past_seq_len : 38
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as

We get a `TSPipeline` after training. Let's print the hyper paramters selected.
Note that `past_seq_len` is the lookback value that is automatically chosen

In [28]:
ts_pipeline.internal.config

{'mean': [470.45753424657534,
  5.633561643835616,
  15.558219178082192,
  0.8333333333333334],
 'scale': [237.54721125499952,
  3.2248983714954504,
  8.827884346441833,
  0.3726779962499649],
 'future_seq_len': 1,
 'dt_col': 'datetime',
 'target_col': 'AvgRate',
 'extra_features_col': None,
 'drop_missing': True,
 'model': 'LSTM',
 'metric': 'mean_squared_error',
 'batch_size': 64,
 'selected_features': '["MONTH(datetime)", "DAY(datetime)", "IS_AWAKE(datetime)"]',
 'lstm_1_units': 16,
 'dropout_1': 0.2,
 'lstm_2_units': 16,
 'dropout_2': 0.3095076432762375,
 'lr': 0.008535167820304366,
 'epochs': 1,
 'past_seq_len': 38}

Use it to do prediction, evaluation or incremental fitting.

In [29]:
pred_df = ts_pipeline.predict(test_df)

RuntimeError: The reset parameter is False but there is no n_features_in_ attribute. Is this estimator fitted?

plot actual and prediction values for `AvgRate` KPI

In [ ]:
# plot the predicted values and actual values
plot_result(test_df, pred_df, dt_col="datetime", value_col="AvgRate", look_back=ts_pipeline.internal.config['past_seq_len'])

Calculate mean square error and the symetric mean absolute percentage error.

In [ ]:
mse, smape = ts_pipeline.evaluate(test_df, metrics=["mse", "smape"])
print("Evaluate: the mean square error is", mse)
print("Evaluate: the smape value is", smape)

You can save the pipeline to file and reload it to do incremental fitting or others.

In [ ]:
# save pipeline file
my_ppl_file_path = ts_pipeline.save("/tmp/saved_pipeline/my.ppl")

You can stop RayOnSpark after auto training.

In [ ]:
# stop
ray_ctx.stop()
sc.stop()

Next, we demonstrate how to do incremental fitting with your saved pipeline file.

First load saved pipeline file.

In [ ]:
# load file
from zoo.zouwu.autots.forecast import TSPipeline
loaded_ppl = TSPipeline.load(my_ppl_file_path)

Then do incremental fitting with `TSPipeline.fit()`.We use validation data frame as additional data for demonstration. You can use your new data frame.


In [ ]:
# we use validation data frame as additional data for demonstration. 
loaded_ppl.fit(val_df, epochs=2)

predict and plot the result after incremental fitting.

In [ ]:
# predict results of test_df
new_pred_df = loaded_ppl.predict(test_df)
plot_result(test_df, new_pred_df, look_back=loaded_ppl.internal.config['past_seq_len'])

Calculate mean square error and the symetric mean absolute percentage error.

In [ ]:
# evaluate test_df
mse, smape = loaded_ppl.evaluate(test_df, metrics=["mse", "smape"])
print("Evaluate: the mean square error is", mse)
print("Evaluate: the smape value is", smape)